In [1]:
# load packages
import ixmp                    # package for ix modeling platform
import message_ix              # package for MESSAGEix model
from message_ix.utils import make_df
import matplotlib.pyplot as plt

<IPython.core.display.Javascript object>

In [6]:
# Loading the platform (and getting conected to a local database)
mp = ixmp.Platform()

In [3]:
model='MESSAGEix South Africa'
scenario_name='added fuel'
baseline = message_ix.Scenario(mp, model=model, 
                               scenario='baseline')

In [7]:
try:
    if len(mp.scenario_list(model=model, scen=scenario_name))>0:
        scenario = message_ix.Scenario(mp, model=model, scenario=scenario_name)
except ValueError:
    # make new scenario
    scenario = baseline.clone(model=model, scenario=scenario_name, annotation='more sophisticated supply chains, attention to units', keep_solution=False)
try:
    scenario.remove_solution()
except Exception:
    pass
#scenario.check_out()

In [8]:
scenario.solve()

In [113]:
# look at time
years = scenario.set('year').tolist()
# find first 
print('first model year', scenario.firstmodelyear)

# Retrieve historic time-steps
history = [y for y in years if y < scenario.firstmodelyear]
future = [y for y in years if y >= scenario.firstmodelyear]
print('the past is' , history)
print('the future is' , future)

first model year 2020
the past is [1955, 1960, 1965, 1970, 1975, 1980, 1985, 1990, 1995, 2000, 2005, 2010]
the future is [2020, 2030, 2040, 2050, 2060, 2070]


In [114]:
# data needs 
demand_vector = {}# includes historic demand in GWa/a 2010

technical_lifetime_for_technologies = {'hb_ammonia':25} # in years
historical_activity_for_technologies =  {} # in GWa/a, by year https://www.indexmundi.com/minerals/?country=za&product=ammonia
historical_new_capacity_for_technologies = {}# in GW\a
investment_cost_for_technologies= {}# in USD/kW by year
fix_cost_for_technologies = {}# in USD/kW/a, by year
var_cost_for_technologies = {}# in USD/kWa, by year

input_matrices_for_technologies = {}# no unit
output_matrices_for_technologies = {}# no unit, usually one 
emission_factors_for_technologies= {}# in MtCO2eq/GWa, by year

# TODO check these
# prfcons
# p_ref

# irrelevant
construction_time_for_technologies = 1 # in years, 
flexibility_factors_for_technologies = 1 #between 0 and 1, by year
##initial_acitivity_up # paraemter to delimit activity change??
#initial_new_capacity_up??  # parameter to delimint capacity change
# relation_activity # in Gwa/a # probably just a placeholder


## Add all the model sets and parameters

### list all technologies that are covered
1. Steam-Methane-Reformation 
    -   Inputs: Methane(2.2t), Heat (5.7MWh), Water (4.9t) https://en.wikipedia.org/wiki/Hydrogen_production
    -   Output: Hydrogen (1.1t)
    -   Emissions: CO2eq (6.0t)
1. Proton-Membrane-Fuel-Cell
    - Inputs: Electricity (39.4 MWh), Water (9.1t) https://en.wikipedia.org/wiki/Hydrogen_production
    - Output: Hydrogen (1.1t)
    - Traditional efficency of 70% today
1. Haber Bosch Ammonia Synthesis
    - Inputs: Nitrogen, Natural gas (61.1 MW / 50kTonnes) (60% energy efficiency)
    - Output: Ammonia
    - Investment cost: 111 000 000 USD / 50kTonnes * a https://www.sciencedirect.com/science/article/pii/S0306261919318227
    - Operating cost: 1 000 000 USD / 50kTonnes * a https://www.sciencedirect.com/science/article/pii/S0306261919318227
    - Lifetime: 25 years https://www.sciencedirect.com/science/article/pii/S0306261919318227
    - Emissions: CO2 but this is accounted for in gas production 0.21 kgCO2eq/kwh https://winnipeg.ca/finance/findata/matmgt/documents/2012/682-2012/682-2012_Appendix_H-WSTP_South_End_Plant_Process_Selection_Report/Appendix%207.pdf
1. Ammonia synthesis with fuel cell https://www.sciencedirect.com/science/article/pii/S0306261919318227

In [115]:
# ammonia activity and demand
hours_per_year = 8760
kWh_to_GWa = 1/1000000/hours_per_year
ammonia_t_to_kwh = 5.17 * 1000
ammonia_t_to_gwa = 5.17 * 0.001 / hours_per_year # https://www.ammoniaenergy.org/articles/round-trip-efficiency-of-ammonia-as-a-renewable-energy-transportation-media/
nitrogen_mass_share = 0.8244
data = {2005: 460000, 2010: 470000} # in t N 
historical_activity_for_technologies['hb_ammonia'] = {2005: data[2005]/nitrogen_mass_share*ammonia_t_to_gwa, 2010: data[2005]/nitrogen_mass_share*ammonia_t_to_gwa}
# ASSUMPTION supply = demand in previous years
demand_vector['ammonia'] = historical_activity_for_technologies['hb_ammonia']
for year in future:
    demand_vector['ammonia'][year] = demand_vector['ammonia'][2010]



In [116]:
# technology assumptions for Haber Bosch Ammonia production
tec = 'hb_ammonia'
technical_lifetime_for_technologies[tec] = 25
# ASSUMPTION 2005 and 2010 half of the capacity needed for 2010 newly installed
historical_new_capacity_for_technologies[tec] = {2005:demand_vector['ammonia'][2010]/5/2, 2010:demand_vector['ammonia'][2010]/5/2}
# ASSUMPTION constant costs
investment_cost_for_technologies[tec] = [110000000/(50000* ammonia_t_to_kwh / hours_per_year)]*len(years) # TODO double check
# ASSUMPTION no fix costs
fix_cost_for_technologies[tec] = [0]*len(years)
var_cost_for_technologies[tec] = [1000000/(50000* ammonia_t_to_kwh / hours_per_year)] *len(years)


In [117]:
tec = 'hb_ammonia'
output_matrices_for_technologies[tec] = 1
input_matrices_for_technologies[tec] = {}
input_matrices_for_technologies[tec]['gas'] = 1/0.6
# TODO add Methane emissions in CO2eq
emission_factors_for_technologies[tec] = 0 # no methane emissions0.21*kWh_to_GWa #MtCO2/

In [118]:
# add ammonia as commodity
country = 'South Africa'
mode='M1'

# Adding required information via MESSAGEix sets
commodity= 'ammonia'
scenario.add_set('commodity', commodity)
# Building the required demand table (python DataFrame)
import pandas as pd
df = pd.DataFrame({'node': country,
                   'commodity': commodity,
                   'level': 'final',
                   'year': [key for key,value in demand_vector[commodity].items()],
                   'value': [value for key,value in demand_vector[commodity].items()],    # cup of coffee per year
                   'unit': 'GWa/a',
                   'time': 'year',
                   'mode':mode,
                   })
scenario.add_par('demand', df)

In [119]:
# add technologies
tec = 'hb_ammonia'
scenario.add_set('technology',tec)
 
# add cost parameters
# installation costs
# add var costs 
# add fix costs
for vintage_year in years:
    base_df = pd.DataFrame({'node_loc':country,'node_dest':country, 'year_vtg':vintage_year,'year_act':years,'mode':mode,'time':'year'})
    df = make_df(base_df,technology=tec,value=pd.Series(investment_cost_for_technologies[tec]), unit= 'USD/kW')
    scenario.add_par('inv_cost',df)

    df = make_df(base_df,technology=tec,value=pd.Series(fix_cost_for_technologies[tec]), unit='USD/kWa')
    scenario.add_par('fix_cost',df)
    
    df = make_df(base_df,technology=tec,value=pd.Series(var_cost_for_technologies[tec]), unit='USD/kWa')
    scenario.add_par('var_cost',df)

In [120]:
# add inputs
tec = 'hb_ammonia'
commodity = 'gas'
#  Levels of the reference energy system or supply chain (primary, secondary, … , useful)
df = pd.DataFrame({'node_loc':country,
                    'technology':tec,
                    'node_origin':country,
                    'mode':mode,
                    'commodity': commodity,
                    'year_vtg':years,
                    'year_act': years,
                    'level':'secondary',
                    'time':'year',
                    'time_origin':'year',
                    'value':input_matrices_for_technologies[tec][commodity],
                    'unit':'-'})

scenario.add_par('input',df)

In [121]:
# Adding data to the parameter "output" for the technology
tec = 'hb_ammonia'
commodity = 'ammonia'

df = pd.DataFrame({'node_loc': country,
                   'node_dest': country,
                   'technology': tec,
                   'commodity': commodity,
                   'level': 'final',
                   'year_vtg': years,  
                   'year_act': years,
                   'mode': mode,
                   'value': output_matrices_for_technologies[tec],
                   'unit': '-',
                   'time': 'year',
                   'time_dest': 'year',
                    })
scenario.add_par('output', df)

In [122]:
#  Add emission factors
tec = 'hb_ammonia'
# we now add CO2 emissions to the coal powerplant
base_emission_factor = {
    'node_loc': country,
    'year_vtg': years,
    'year_act': years,
    'mode': mode,
    'unit': 'tCO2/kWa',
}

emission_factor = make_df(base_emission_factor, technology= tec, emission= 'CO2', value = emission_factors_for_technologies[tec])
scenario.add_par('emission_factor', emission_factor)

In [123]:
base_technical_lifetime = {
    'node_loc': country,
    'year_vtg': years,
    'unit': 'y',
}
for tec, val in technical_lifetime_for_technologies.items():
    df = make_df(base_technical_lifetime, technology=tec, value=val)
    scenario.add_par('technical_lifetime', df)

In [124]:

scenario.commit('added ammonia supply and demand')

In [125]:
# artifically add hydrogen demand
scenario.check_out()
# add ammonia as commodity
country = 'South Africa'
mode='M1'

# Adding required information via MESSAGEix sets
commodity= 'hydrogen'
scenario.add_set('commodity', commodity)
# Building the required demand table (python DataFrame)
import pandas as pd
df = pd.DataFrame({'node': country,
                   'commodity': commodity,
                   'level': 'final',
                   'year': [key for key,value in demand_vector['ammonia'].items()],
                   'value': [value for key,value in demand_vector['ammonia'].items()],    # cup of coffee per year
                   'unit': 'GWa/a',
                   'time': 'year',
                   'mode':mode,
                   })
scenario.add_par('demand', df)
scenario.commit('hydrogen added')

In [132]:
scenario.discard_changes()
scenario.solve()

In [133]:
# Objective value (why is it zero?)
scenario.var('OBJ')

{'lvl': 22625842.0, 'mrg': 0.0}

In [ ]:
# TODO add
# flexibility_factors_for_technologies = #between 0 and 1, by year
#construction_time_for_technologies = # in years, 
#historical_activity_for_technologies = # in GWa/a, by year https://www.indexmundi.com/minerals/?country=za&product=ammonia
#historical_new_capacity_for_technologies = # in GW\a

#investment_cost_for_technologies= # in USD/kW by year
#technical_lifetime_for_technologies = # in years

In [ ]:
#TODO # adjust emission scaling for methane
# TODO unit and sign of emissions

In [9]:
# close platform connection
mp.close_db()